In [195]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [196]:
class MobilePhone:
    def __init__(self, model, price, antutu_score, dxomark, ppi, battery_capacity):
        self.model = model
        self.price = price
        self.antutu_score = antutu_score
        self.dxomark = dxomark
        self.ppi = ppi
        self.battery_capacity = battery_capacity

    def __str__(self):
        return f"Model: {self.model}, Price ($): {self.price}, Antutu Score: {self.antutu_score}, DXOMARK: {self.dxomark}, PPI: {self.ppi}, Battery Capacity (mAh): {self.battery_capacity}"

    def __eq__(self, other):
        if not isinstance(other, MobilePhone):
            return NotImplemented
        return (self.model == other.model and
                self.price == other.price and
                self.antutu_score == other.antutu_score and
                self.dxomark == other.dxomark and
                self.ppi == other.ppi and
                self.battery_capacity == other.battery_capacity)


In [197]:
#wczytaj smartfony i utwórz liste z objektami klasy MobilePhone
data = pd.read_csv("smartfony.csv")
phones = []
phones = [MobilePhone(row['Model'], int(row['Cena ($)']), int(row['Antutu Score']), 
          int(row['DXOMARK']), int(row['ppi']), int(row['pojemność baterii (mAh)'])) for index, row in data.iterrows()]



In [198]:
#wyswietl telefony
for phone in phones:
    print(phone)

Model: Huawei Mate 40 Pro, Price ($): 907, Antutu Score: 637017, DXOMARK: 143, PPI: 450, Battery Capacity (mAh): 4360
Model: Google Pixel 7 Pro, Price ($): 899, Antutu Score: 788707, DXOMARK: 147, PPI: 512, Battery Capacity (mAh): 5000
Model: Honor Magic4 Ultimate, Price ($): 1211, Antutu Score: 1051488, DXOMARK: 147, PPI: 460, Battery Capacity (mAh): 4600
Model: Apple iPhone 14 Pro Max, Price ($): 1299, Antutu Score: 960137, DXOMARK: 146, PPI: 460, Battery Capacity (mAh): 4323
Model: Xiaomi 12S Ultra, Price ($): 908, Antutu Score: 1061590, DXOMARK: 136, PPI: 522, Battery Capacity (mAh): 4860
Model: Samsung Galaxy S22 Ultra, Price ($): 1199, Antutu Score: 907693, DXOMARK: 135, PPI: 500, Battery Capacity (mAh): 5000
Model: ROG Phone 6 PRO, Price ($): 999, Antutu Score: 1116637, DXOMARK: 81, PPI: 395, Battery Capacity (mAh): 6000
Model: Motorola Edge 30 Pro, Price ($): 699, Antutu Score: 979697, DXOMARK: 101, PPI: 393, Battery Capacity (mAh): 4800
Model: Galaxy Z Fold4, Price ($): 1799, 

In [199]:
class Criterion:
    def __init__(self, name, type, weight, indiff, pref, veto):
        self.name = name
        self.type = type
        self.weight = weight
        self.indiff = indiff
        self.pref = pref
        self.veto = veto

# Initialize criteria
criteria = [
    Criterion("price", "cost", 2, 50, 200, 600),
    Criterion("antutu", "gain", 4, 100000, 300000, 500000),
    Criterion("dxomark", "gain", 4, 10, 20, 40),
    Criterion("dpi", "gain", 2, 20, 50, 100),
    Criterion("battery", "gain", 3, 400, 1200, 2000)
]

In [200]:
# Creating the matrix of concordance
concordance = np.ones((len(phones), len(phones)))

# The mapping from phone attributes to the criteria list indices
attribute_mapping = {
    "price": lambda x: x.price,
    "antutu": lambda x: x.antutu_score,
    "dxomark": lambda x: x.dxomark,
    "dpi": lambda x: x.ppi,
    "battery": lambda x: x.battery_capacity
}

for i, phone1 in enumerate(phones):
    for j, phone2 in enumerate(phones):
        cis = []
        dis = []
        for criterion in criteria:
            value1 = attribute_mapping[criterion.name](phone1)
            value2 = attribute_mapping[criterion.name](phone2)
            
            if criterion.type == "gain":
                if value1 - value2 >= -criterion.indiff:
                    cis.append(1)
                elif value1 - value2 < -criterion.pref:
                    cis.append(0)
                else:
                    cis.append((criterion.pref - (value2 - value1)) / (criterion.pref - criterion.indiff))
                
                if value1 - value2 <= -criterion.veto:
                    dis.append(1)
                elif value1 - value2 >= -criterion.pref:
                    dis.append(0)
                else:
                    dis.append(((value2 - value1) - criterion.pref) / (criterion.veto - criterion.pref))

            else:  # Assuming 'cost' type here
                if value1 - value2 <= criterion.indiff:
                    cis.append(1)
                elif value1 - value2 > criterion.pref:
                    cis.append(0)
                else:
                    cis.append((criterion.pref - (value1 - value2)) / (criterion.pref - criterion.indiff))

                if value1 - value2 >= criterion.veto:
                    dis.append(1)
                elif value1 - value2 <= criterion.pref:
                    dis.append(0)
                else:
                    dis.append((criterion.veto - (value1 - value2)) / (criterion.veto - criterion.pref))

        c = sum(ci * criterion.weight for ci, criterion in zip(cis, criteria)) / sum(criterion.weight for criterion in criteria)
        sigma = c

        for d in dis:
            if d > c:
                if c == 1:
                    print("c = 1")
                sigma *= (1 - d) / (1 - c)
        
        concordance[i][j] = round(sigma, 2)

print(concordance)

[[1.   0.74 0.73 0.73 0.51 0.58 0.11 0.03 0.73 0.6  0.63 0.64 0.87 0.87
  0.62 0.41 0.6  0.38 0.62 0.86]
 [1.   1.   0.78 0.9  0.77 0.97 0.58 0.75 0.87 0.84 1.   1.   0.87 0.87
  0.73 0.75 0.77 0.82 0.92 0.93]
 [0.87 0.73 1.   1.   0.71 0.91 0.06 0.87 1.   0.87 0.87 0.73 0.   0.
  0.87 0.   0.   0.73 0.73 0.87]
 [0.87 0.66 0.97 1.   0.7  0.8  0.36 0.   1.   0.87 0.8  0.66 0.   0.
  0.8  0.   0.   0.66 0.73 0.8 ]
 [1.   0.97 0.97 1.   1.   1.   0.81 0.15 1.   1.   1.   1.   0.87 0.87
  0.95 0.87 0.87 0.9  0.91 0.92]
 [0.87 0.81 0.89 0.97 0.79 1.   0.57 0.87 1.   1.   0.98 0.84 0.   0.
  0.8  0.   0.   0.84 0.87 0.87]
 [0.   0.   0.   0.   0.   0.   1.   0.38 0.   0.   0.   0.   0.87 0.
  0.87 0.6  0.87 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.75 1.   0.73 0.   0.   0.   0.49 0.64
  1.   0.96 0.87 0.   0.   0.73]
 [0.   0.   0.23 0.23 0.   0.   0.   0.   1.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.  ]
 [0.87 0.79 0.89 0.95 0.87 0.96 0.47 0.   1.   1.   1.   0.87 0. 

In [201]:
def descending_destilation(matrix, variants, upper_treshold=None, depth=0):
    ans = []
    depth += 1

    while True:
        n = matrix.shape[0]
        matrix_copy = deepcopy(matrix)

        if upper_treshold is None:
            mask = np.eye(n, dtype=bool)
            masked_arr = np.ma.masked_array(matrix, mask)
            upper_treshold = np.max(masked_arr)

        if upper_treshold == 0:
            if len(ans) == 1:
                return ans
            else:
                ans.extend(variants)  # Use extend to add each element as separate if needed.
                return ans
            
        if len(variants) == 0:
            return ans
        if len(variants) == 1:
            ans.extend(variants)  # Change here to use extend or append properly
            return ans
        
        s = -0.15 * upper_treshold + 0.3
        lower_treshold = np.max(matrix[matrix < upper_treshold - s]) if np.any(matrix < upper_treshold - s) else 0

        for i in range(n):
            for j in range(n):
                if matrix[i][j] > lower_treshold and matrix[i][j] > matrix[j][i] + s:
                    continue
                else:
                    matrix_copy[i][j] = 0

        strength = np.array([np.count_nonzero(row) for row in matrix_copy])
        weakness = np.array([np.count_nonzero(column) for column in matrix_copy.T])
        quality = strength - weakness

        if np.count_nonzero(quality == np.max(quality)) == 1:
            max_index = np.argmax(quality)
            ans.append(variants[max_index])
            variants.pop(max_index)
            matrix = np.delete(matrix, max_index, axis=0)
            matrix = np.delete(matrix, max_index, axis=1)
        else:
            max_indices = np.where(quality == np.max(quality))[0].tolist()
            internal_matrix = deepcopy(matrix_copy[max_indices][:, max_indices])
            internal_variants = [variants[i] for i in max_indices]

            internal_ans = descending_destilation(internal_matrix, internal_variants, lower_treshold, depth)
            internal_indices = []
            print("Incomparability!")
            for phone in internal_ans:
                print(phone)
            for variant in internal_ans:
                try:
                    internal_indices.append(variants.index(variant))
                except ValueError:
                    continue  # Skip if variant not found

            internal_indices = sorted(internal_indices, reverse=True)
            for internal_index in internal_indices:
                variants.pop(internal_index)

            matrix = np.delete(matrix, np.array(internal_indices), axis=0)
            matrix = np.delete(matrix, np.array(internal_indices), axis=1)

            ans.extend(internal_ans)  # Ensure that internal_ans is extended, not just appended.

        if matrix.shape[0] == 0:
            return ans




des_dest = descending_destilation(concordance, phones)

Incomparability!
Model: Honor Magic4 Ultimate, Price ($): 1211, Antutu Score: 1051488, DXOMARK: 147, PPI: 460, Battery Capacity (mAh): 4600
Model: Samsung Galaxy S22 Ultra, Price ($): 1199, Antutu Score: 907693, DXOMARK: 135, PPI: 500, Battery Capacity (mAh): 5000
Incomparability!
Model: Honor Magic4 Ultimate, Price ($): 1211, Antutu Score: 1051488, DXOMARK: 147, PPI: 460, Battery Capacity (mAh): 4600
Model: Samsung Galaxy S22 Ultra, Price ($): 1199, Antutu Score: 907693, DXOMARK: 135, PPI: 500, Battery Capacity (mAh): 5000
Incomparability!
Model: Vivo X80 Pro, Price ($): 1299, Antutu Score: 1005160, DXOMARK: 134, PPI: 517, Battery Capacity (mAh): 4700
Model: realme GT2 Pro, Price ($): 749, Antutu Score: 956516, DXOMARK: 106, PPI: 526, Battery Capacity (mAh): 5000
Incomparability!
Model: Vivo X80 Pro, Price ($): 1299, Antutu Score: 1005160, DXOMARK: 134, PPI: 517, Battery Capacity (mAh): 4700
Model: realme GT2 Pro, Price ($): 749, Antutu Score: 956516, DXOMARK: 106, PPI: 526, Battery C

In [202]:
for phone in des_dest:
    print(phone)

Model: Xiaomi 12S Ultra, Price ($): 908, Antutu Score: 1061590, DXOMARK: 136, PPI: 522, Battery Capacity (mAh): 4860
Model: Google Pixel 7 Pro, Price ($): 899, Antutu Score: 788707, DXOMARK: 147, PPI: 512, Battery Capacity (mAh): 5000
Model: iQOO 9 Pro, Price ($): 757, Antutu Score: 858575, DXOMARK: 127, PPI: 518, Battery Capacity (mAh): 4700
Model: OnePlus 10 Pro, Price ($): 899, Antutu Score: 865589, DXOMARK: 122, PPI: 525, Battery Capacity (mAh): 5000
Model: Honor Magic4 Ultimate, Price ($): 1211, Antutu Score: 1051488, DXOMARK: 147, PPI: 460, Battery Capacity (mAh): 4600
Model: Samsung Galaxy S22 Ultra, Price ($): 1199, Antutu Score: 907693, DXOMARK: 135, PPI: 500, Battery Capacity (mAh): 5000
Model: Vivo X80 Pro, Price ($): 1299, Antutu Score: 1005160, DXOMARK: 134, PPI: 517, Battery Capacity (mAh): 4700
Model: realme GT2 Pro, Price ($): 749, Antutu Score: 956516, DXOMARK: 106, PPI: 526, Battery Capacity (mAh): 5000
Model: Huawei Mate 40 Pro, Price ($): 907, Antutu Score: 637017, 